In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import pydicom
from torchvision import transforms
from collections import defaultdict

class MRIDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        self.bags = defaultdict(list)
        for _, row in self.data.iterrows():
            self.bags[row['SeriesInstanceUID']].append(row)
    
    def __len__(self):
        return len(self.bags)
    
    def __getitem__(self, idx):
        bag_id = list(self.bags.keys())[idx]
        bag = self.bags[bag_id]
        
        images = []
        for item in bag:
            img_path = os.path.join(self.root_dir, item['SeriesInstanceUID'])
            for img_file in os.listdir(img_path):
                if img_file.endswith('.dcm'):
                    dcm = pydicom.dcmread(os.path.join(img_path, img_file))
                    img = dcm.pixel_array.astype(np.float32)
                    img = (img - img.min()) / (img.max() - img.min())
                    if self.transform:
                        img = self.transform(img)
                    images.append(img)
        
        label = bag[0]['prediction']
        body_part = bag[0]['Body Part Examined']
        series_desc = bag[0]['Series Description']
        
        return torch.stack(images), label, body_part, series_desc

class AttentionMIL(nn.Module):
    def __init__(self, num_classes=2):
        super(AttentionMIL, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.attention = nn.Sequential(
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        x = x.squeeze(0)
        H = self.feature_extractor(x)
        H = H.view(-1, 128)
        
        A = self.attention(H)
        A = torch.transpose(A, 1, 0)
        A = nn.functional.softmax(A, dim=1)
        
        M = torch.mm(A, H)
        Y_prob = self.classifier(M)
        Y_hat = torch.argmax(Y_prob, dim=1)
        
        return Y_prob, Y_hat, A

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (data, targets, _, _) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        
        outputs, _, _ = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    accuracy = 100. * correct / total
    return running_loss / len(train_loader), accuracy

def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (data, targets, _, _) in enumerate(test_loader):
            data, targets = data.to(device), targets.to(device)
            
            outputs, _, _ = model(data)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    accuracy = 100. * correct / total
    return running_loss / len(test_loader), accuracy

batch_size = 1
num_epochs = 30
learning_rate = 0.001
csv_file = '/kaggle/input/editedcsv/FinalMetaData.csv'
root_dir = '/kaggle/input/iaaa-mri-challenge/data/'

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = MRIDataset(csv_file, root_dir, transform=transform)
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AttentionMIL().to(device)

criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0]).to(device))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    print('-----------------------------')

torch.save(model.state_dict(), 'mil_model.pth')

Epoch 1/30:
Train Loss: 0.3942, Train Acc: 87.58%
Test Loss: 0.3813, Test Acc: 87.24%
-----------------------------
Epoch 2/30:
Train Loss: 0.3879, Train Acc: 87.58%
Test Loss: 0.3844, Test Acc: 87.24%
-----------------------------
Epoch 3/30:
Train Loss: 0.3827, Train Acc: 87.58%
Test Loss: 0.3851, Test Acc: 87.24%
-----------------------------
Epoch 4/30:
Train Loss: 0.3804, Train Acc: 87.58%
Test Loss: 0.3839, Test Acc: 87.24%
-----------------------------
Epoch 5/30:
Train Loss: 0.3802, Train Acc: 87.58%
Test Loss: 0.3812, Test Acc: 87.24%
-----------------------------
Epoch 6/30:
Train Loss: 0.3828, Train Acc: 87.58%
Test Loss: 0.3824, Test Acc: 87.24%
-----------------------------
Epoch 7/30:
Train Loss: 0.3786, Train Acc: 87.58%
Test Loss: 0.3822, Test Acc: 87.24%
-----------------------------
Epoch 8/30:
Train Loss: 0.3767, Train Acc: 87.58%
Test Loss: 0.3831, Test Acc: 87.24%
-----------------------------
Epoch 9/30:
Train Loss: 0.3775, Train Acc: 87.58%
Test Loss: 0.3828, Tes